In [2]:
#encoding=utf-8
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
# import lightgbm as lgb
from sklearn import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import MultinomialNB

import pickle
from sklearn.model_selection import train_test_split
from all_model_py import CutDebt, IDClassifier, IfKnowDebtor, Installment, WillingToPay, ConfirmLoan

# import sys
# sys.path.append('../CutDebt/')
# sys.path.append('../IDClassifier/')
# sys.path.append('../IfKnowDebtor/')
# sys.path.append('../Installment/')
# sys.path.append('../SetDueDay/')
# sys.path.append('../WillingToPay/')

# from CutDebt_py import CutDebt
# from IDClassifier_py import IDClassifier
# from IfKnowDebtor_py import IfKnowDebtor
# from Installment_py import Installment
# from SetDueDay_py import SetDueDay
# from WillingToPay_py import WillingToPay

In [3]:
'''
specific_accuracy = {} #record each parameted of the corresponding zero count
accuracy=[] #record the threshold with highest accuracy
result #the average probablity of three models
label_result  #record the label with highest probablity

'''




model_list = ['CutDebt','IDClassifier','IfKnowDebtor','Installment','ConfirmLoan','WillingToPay']
func_list = [CutDebt,IDClassifier,IfKnowDebtor,Installment,ConfirmLoan,WillingToPay]

accuracy=[]
ind = 0
specific_accuracy = {} #record each parameted of the corresponding zero count
for each_model in model_list:
    
    print(each_model)
    
    # load data
    path = '../../data/{}/'
    data = pd.read_csv(path.format(each_model) + 'cleaned_mock_up_data.csv', encoding='utf8')
    
    
    [data_train,data_test] = train_test_split(data, test_size = 0.1, shuffle=True)
    print(data.loc[0,:].label)
    
#     data_train = data.get_chunk(5)
#     data_test = pd.read_csv(path.format(each_model) + 'cleaned_mock_up_data.csv', encoding='utf8')
    
#     data_train = 
#     data_test = 
    
    # get tfidf
    phrase_vectorizer = TfidfVectorizer(ngram_range=(1,3),
                                    strip_accents='unicode', 
                                    max_features=100000, 
                                    analyzer='word',
                                    sublinear_tf=True,
                                    token_pattern=r'\w{1,}')

    print('fitting phrase')
    phrase_vectorizer.fit(data_train.split_text)

    print('transform phrase')
    phrase = phrase_vectorizer.transform(data_train.split_text)

    
    # linear svc
    l_svc = LinearSVC()
    lsvc = CalibratedClassifierCV(l_svc) 
    lsvc.fit(phrase, data_train.label)
    
    
    # logistic
    log_r = LogisticRegression()
    log_r.fit(phrase, data_train.label)
    
    
    # Naive Bayes
    naive_b = MultinomialNB()
    naive_b.fit(phrase, data_train.label)
    
    
    print('finish training')
    
    print('start test')
    
    text = phrase_vectorizer.transform(data_test.split_text)
    temp1 = np.array(lsvc.predict_proba(text))
    temp2 = np.array(log_r.predict_proba(text))
    temp3 = np.array(naive_b.predict_proba(text))
    result = np.zeros_like(temp1)
    for i in range(len(temp1)):
        result[i] = (temp1[i] + temp2[i] + temp3[i])/3
    label_result = np.argmax(result,axis=1)
    count = 0
    
    
    test_index = len(data_test) - 1
    test_label = data_test
    max = 0 #zero count
    accuracy_max = 0 #keep the most accuracte value
    c = 0.2 
    for x in range(0,60):
        ind = (np.max(result, axis =1) < c)
        result[ind,-1] = 1 
        label_result = np.argmax(result,axis = 1)
        temp = data_test['label'].values - label_result
        accuracy_temp = temp.tolist().count(0) #current zero count
        specific_accuracy[each_model+str(c)] = accuracy_temp
        if accuracy_temp > max:
            max = accuracy_temp
            accuracy_max = c       
        c += 0.01
        
    accuracy.append(accuracy_max)
    
        
    
                
    
    
#     for threhold
    
    # save model
#     save_path = '../../savedModel/{}/'
#     # save tfidf
#     pickle.dump(phrase_vectorizer, open(save_path.format(each_model) + "tfidf.pickle", "wb"))
#     # save linear svc
#     pickle.dump(lsvc, open(save_path.format(each_model) + "LinearSVC.pickle", "wb"))
#     # save logistic
#     pickle.dump(log_r, open(save_path.format(each_model) + "Logistic.pickle", "wb"))
#     # save lightGBM
# #     pickle.dump(lgbm_model, open(save_path.format(each_model) + "Lgbm.pickle", "wb"))
#     # save naive bayes
#     pickle.dump(naive_b, open(save_path.format(each_model) + "nb.pickle", "wb"))

    
    
    
#     func = func_list[ind]
#     result = func(svc=lsvc, logistic=log_r, nb=naive_b, tfidf=phrase_vectorizer, jieba_path='../WordCut/userdict.txt')
#     pickle.dump(result, open(save_path.format(each_model) + each_model + '.pickle', "wb"))
#     ind = ind + 1
    
    
    
    

CutDebt
0
fitting phrase
transform phrase
finish training
start test
IDClassifier
0
fitting phrase
transform phrase
finish training
start test
IfKnowDebtor
2
fitting phrase
transform phrase
finish training
start test
Installment
0
fitting phrase
transform phrase
finish training
start test
ConfirmLoan
1
fitting phrase
transform phrase
finish training
start test
WillingToPay
2
fitting phrase
transform phrase
finish training
start test


In [4]:
'''
specific_accuracy = {} #record each parameted of the corresponding zero count
accuracy=[] #record the threshold with highest accuracy
result #the average probablity of three models
label_result  #record the label with highest probablity

'''

'\nspecific_accuracy = {} #record each parameted of the corresponding zero count\naccuracy=[] #record the threshold with highest accuracy\nresult #the average probablity of three models\nlabel_result  #record the label with highest probablity\n\n'